# Preprocessing

* <span style="color:green">Alignment of neuronal data with behaviour</span>
* <span style="color:green">Choice of basis subtraction </span>
    * <span style="color:green">Raw </span>
    * <span style="color:green">Bursts via 20% quantile</span>
    * <span style="color:red">ZScore by cell-average</span>
    * <span style="color:red">ZScore by initial period</span>
    * <span style="color:red">Deconvolution</span>

# Aims

Cells 

### Temporal coding hypotheses
* **HTCRAND**: Cells fire stochastically. While firing rate may be context, dependent, the firing times are random
* **HTCTIME**: Cells fire at specific times relative to the behavioural intervals
* **HTCSYNC**: Several cells are likely to fire simultaneously, while not being time cells
* **HTCSEQ**: One cell is likely to fire a specific interval away from another



# Exploratory Procedure

## Understanding Neuronal Correlates of Performance and Direction

General dataset has structure (Mouse, Session, Trial, Neuron, Timestep). Each trial is associated with 4 possible behavioural outcomes (Hit/Miss) $\otimes$ (L/R). Goal is to predict behavioural outcome from data

### Concerns
* **Sample size**: Have ~100 samples per mouse. This is only sufficient for fairly strong effects.
* **Imbalance**: (Hit/Miss) have strong imbalance of ~100 vs ~10 trials. Impossible to demonstrate significant difference in predictive power. Impossible to perform cross-validation on so few datapoints.
* **Overfitting**: If multivariate representation is used, predictor of behavioural outcome needs to be trained. Poor cross-validated prediction may be attributed to overfitted predictor as well as true lack of relationship. Predictors with low sample size and high dimension are likely to overfit.
* **Temporal match**: Trials have different duration. In case temporal activity is taken into account, assumption needs to be made about time-correspondence between individual trials.
* **Sparse noisy signal**: Signal is sparse and nonlinear, with highly variable SNR that is frequently low. Without deconvolution, each timestep has to be treated as potentially containing information. Since spikes are rare in raw data, their effect likely will not appear significant.
* **Poor temporal hypotheses**: What do we actually expect to see? Is a spike guaranteed to be part of a sequence? Are intervals between spikes in a sequence uniform? How long are they? How are the sequences mediated? 
* **Coarse temporal resolution**: Is the expected sequence faster or slower than the sampling timescale. 

### Predictor Hypotheses
* **HP1**: `avg(data, axis={Neurons, Time})`
* **HP2**: `avg(data[Neuron], axis={Time})` - Each neuron individually is a predictor
* **HP3**: `data[Neuron, Time]` - Time-coding 
* **HP3**: `avg(data, axis={Time})` - Population coding of neuronal average rates

### Temporal matching hypotheses
* **HTSTAT**: Semistatic hypothesis. Individual timesteps matter, but their order does not. Problem reduces to comparing empirical probability distributions in neuronal space.
* **HTDYN**: Dynamic hypothesis. Order of timesteps matters. Assumption on temporal relationship (e.g. MAR(1)) is required to proceed.

More specifically
* **HTS1D**: Predictor is empirical PDF of 1 neuron. Plausible metrics are Mean, Variance
* **HTSND**: Predictor is empirical PDF of a subset of neurons.
    * **HTSND1**: Most of variance is across different axis. Compute distance between subspaces given by small number of PCA's
    * **HTSND2**: Different amount of inter-neuronal relationship. Plausible metrics: explained variance for first few PCA's, `avg(abs(offdiag(corr)))`, ND Entropy, Total Correlation
* **HTDSCALED**: Different trials are stretched versions of the base trial. Plausible metrics include `Orderability`, `LegendreBasis`
    * `TODO` For each neuron, test trial-average variance for stretched vs truncated. Try evaluate what captures better. Design null hypothesis where neurons fire completely stochastically (e.g. random cycle each trial). 
    * `TODO` Plot avg scaled activity L vs R, Corr vs Miss, see if looks different
* **HTDEXT**: System follows a certain ODE, which simply runs longer. Would be characterized by persistent time-relations between variables. Plausible metrics include MAR efficiency, PI/TE. Expect estimation to be poor on non-deconvolved data because of sparsity and high noise
* **HTDTRANS**: Neurons perform similar activity, but padded by arbitrary waiting time before and after. Cell-wise cross-correlation of trials (should work for uneven trials too)
* **Other assumptions**: Orderability, Trunc PSD, PhaseSpace Clustering Distance

# Results

* Bimodality explained by two types of mice
* Corr plot suggests there is mild clustering
* ord plot suggests exist highly orderable cells -> perhaps I try to plot stretch over time
* Prediction
    - (CL/CR:ALL) so far unsuccessful
    - (CL/CR:Mice) seems significant on m065 for mean, corr, timing. For others non-significant
    - (CL/ML:Mice) Is impossible to do. There are not enough mistake repetitions to split data into training vs test
    - (CL/ML:ALL) Still need to do

# TODO

Priority-High
* Calc confusion diagonal for deconvolved data
* Read Yaro description of PCA on WhatsApp
* Put all results into presentation, add text description
* Decide on actual metrics of interest
* Sweep metrics of interest over regions
* Bin Maintenance into Start|Mid|End, calc sequences on 3 bins

PRIORITY-MED
* Sanity checks for mesostat metrics
* Plot correct vs mistake curves like Maria
* Construct separate test tables for direction and performance
* (med) Impl filter reward cells  -> ask maria where exactly they were again?
    * Event 3 :: [-0.5, +1.0];  Event 12 :: [-0.5, +1.0];  (try union and each separately)
* Orderability
    * Attempt filter out reward cells from Maria's data
    * Impl pvalue for bivariate orderability
* Test individual cells for LR by sweeping over time
    * What do we expect to gain from this? It is a lot of plots? How to test significance?


PRIORITY-LOW
* (med) Inverter - for each trial, crop phases to the one that is shortest
* (easy) Compute derivative metrics (TC, TPI), see if their var is larger than for H,PI
* (yaro suggest) Try product of orderability(session) * mean(trial)
* Attempt to fix classifiers.
    * Classifiers that correct for overfitting
* Test why PI has high significance but very close values. Try eg scatter plot of Encoding vs Maintenance
* Inverter - Fixing non-comparable nSamples for diff modes
    * For each trial, pick min phase, truncate other
    * Maria suggests to truncate Maintenance towards end
* Investigate bimodality causes (H, AvgCorr)
    * Redo with split by direction (left more important)
    * Look at m068
* Mega-clustering
    * Resample/truncate all to same timescale
    * T-SNE on all dims (Channel * Time) over trials
    * Color by CM * LR